In [ ]:
from copy import deepcopy

import matplotlib.pyplot as plt
import numpy as np
import pyvista as pv
from matplotlib.ticker import FormatStrFormatter

from ogstools.propertylib import MatrixProperty, VectorProperty, defaults

pvds = ["", ""]
params = [defaults.temperature]

ts = 100
timeseries = pv.PVDReader(pvds[0])
timeseries.set_active_time_point(ts)
mesh0 = timeseries.read()[0]
meshes = []
e_lengths = []
for pvd in pvds:
    mesh_temp = deepcopy(mesh0)
    timeseries = pv.PVDReader(pvd)
    timeseries.set_active_time_point(ts)
    mesh = timeseries.read()[0]
    mesh_temp = mesh_temp.sample(mesh, pass_cell_data=False)
    e_lengths += [np.mean(np.sqrt(mesh.compute_cell_sizes().cell_data["Area"]))]
    meshes += [mesh_temp]

In [ ]:
rich_ex = deepcopy(meshes[0])
r = 2  # refinement ratio
for param in params:
    f1 = meshes[-1].point_data[param.data_name]
    f2 = meshes[-2].point_data[param.data_name]
    rich_ex.point_data[param.data_name] = f1 + (f1 - f2) / (r * r - 1)

In [ ]:
for param in params:
    _p_val = (
        param.magnitude
        if isinstance(param, (VectorProperty, MatrixProperty))
        else param
    )
    err = [
        np.linalg.norm(
            _p_val.values(rich_ex.point_data[param.data_name])
            - _p_val.values(mesh.point_data[param.data_name]),
            axis=0,
            ord=2,
        )
        / np.linalg.norm(
            _p_val.values(rich_ex.point_data[param.data_name]),
            axis=0,
            ord=2,
        )
        for mesh in meshes
    ]
    print(max(_p_val.values(rich_ex.point_data[param.data_name])))
    # https://www.sd.rub.de/downloads/Convergence_FEM.pdf
    fig, ax = plt.subplots(dpi=200, figsize=[7, 2.5], facecolor="white")
    ax.loglog(e_lengths, err, "-o")
    p1 = [(err[0] / e_lengths[0]) * el**1 for el in e_lengths]
    ax.loglog(e_lengths, p1, "--", c="k")
    p2 = [(err[0] / e_lengths[0] ** 2) * el**2 for el in e_lengths]
    ax.loglog(e_lengths, p2, ":", c="k")
    # ax.set_title(title, loc="center", y=1.02)
    ax.set_xlabel("mean element length / m")
    ax.set_ylabel(f"{param.output_name} error / -")
    ax.legend(["L2 Norm", "p=1", "p=2"])
    ax.grid(True, "major", "both", alpha=0.5)
    ax.grid(True, "minor", "both", alpha=0.1)

    ax.xaxis.set_minor_formatter(FormatStrFormatter("%.0f"))
    ax.xaxis.set_major_formatter(FormatStrFormatter("%.0f"))

    # x_ticks = range(len(labels[:-1]))
    # x_tick_labels = [l.replace(", ", "\n") for l in labels[:-1]]
    # ax.set_xticks(x_ticks, x_tick_labels, fontsize=7)

    plt.tight_layout()
    # plt.savefig(fig_path + f"Convergence_{param.output_name}")
    plt.show()
    plt.close()
    fig = None